In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)
submission_df = pd.read_csv('submissions.csv')
submission_df.head()

,id,kind,category,created_utc,author,name,subreddit_id,subreddit_subscriber,subreddit,title,selftext,upvote_ratio,url,num_comments,ups,downs,total_awards_received,score,created,num_crossposts
0,slzdzs,t3,NaN,1.644162e+09,Independent_Bee_4468,t3_slzdzs,t5_2qs0k,51514,Home,What is this mud-like lump that recently appea...,NaN,0.50,https://www.reddit.com/gallery/slzdzs,28,0,0,0,0,1.644162e+09,0
1,qrti73,t3,NaN,1.636660e+09,TopSpite9394,t3_qrti73,t5_2qs0k,51514,Home,Any idea what this is? Sorry if this is a dumb...,NaN,0.60,https://www.reddit.com/gallery/qrti73,24,15,0,0,15,1.636660e+09,0
2,p1se4g,t3,NaN,1.628612e+09,villanegg,t3_p1se4g,t5_2qs0k,51514,Home,I just bought my first flat. First pic!,NaN,0.53,https://i.redd.it/ygtww9pv1kg71.jpg,6,1,0,0,1,1.628612e+09,0
3,oeiaps,t3,NaN,1.625525e+09,Cristheideaman,t3_oeiaps,t5_2qs0k,51514,Home,Exterior project my family completed follow on...,NaN,0.50,https://v.redd.it/sd1smhg84h971,0,0,0,0,0,1.625525e+09,0
4,nadi4r,t3,NaN,1.620784e+09,Nahughes121989,t3_nadi4r,t5_2qs0k,51514,Home,Loving my new living room setup. Just need a p...,NaN,0.59,https://i.redd.it/gt880tx8hly61.jpg,11,6,0,0,6,1.620784e+09,0


In [3]:
submission_df = submission_df.dropna(subset=['title'])
group_submission = submission_df[['subreddit', 'title']].groupby('subreddit')
submission_corpus = group_submission['title'].apply(lambda txt: ". ".join(txt)).reset_index()
submission_corpus.head()

,subreddit,title
0,2007scape,I was trying to record this potential scammer ...
1,3Dprinting,Official Response from Cults regarding the u/B...
2,AmItheAsshole,AITA for having my 18 year old daughter pay fo...
3,AskMen,What is your experience with quiet girls?. Wha...
4,AskReddit,Would you refrain from masturbation or sex for...


In [4]:
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
STOPWORDS = set(stopwords.words('english'))

def data_cleaning(txt):
    txt = txt.translate(str.maketrans('', '', string.punctuation))
    txt = " ".join([word for word in txt.split() if word not in STOPWORDS])
    pos_tagged_text = nltk.pos_tag(txt.split())
    txt = " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
    return txt
    
submission_corpus['title'] = submission_corpus["title"].apply(lambda x: data_cleaning(x))
submission_corpus.head()

,subreddit,title
0,2007scape,I try record potential scammer actually glitch...
1,3Dprinting,Official Response Cults regard uBigbore729 cas...
2,AmItheAsshole,AITA 18 year old daughter pay feminine hygiene...
3,AskMen,What experience quiet girl What tell partner o...
4,AskReddit,Would refrain masturbation sex 365 day 1000000...


# 1. Data Preprocessing

Text corpus depends on the application domain, so it should be contextualised since the window of context will determine what words are considered to be related. Following are the main processes that can improve the results of topic modeling.
- Text cleaning
- Tokenizations
- Stop word removal
- Lemmatization
- Part of speech tagging
- TF-IDF 

PS. All the decision made below on data preprocessing are based on multiple failed attempt on extracting important topics from LDA model. Hence, I've create another file for these failed attempts and clarify the findings here: https://github.com/leslieyuandev/.

### Frequency Filter - Remove words that appear  in > 10% of subreddits

### Frequency Filter - Remove words that appear < 20 subreddits

### TF-IDF 

In [27]:
tfv = TfidfVectorizer(min_df=3, 
                      max_features=None,
                      strip_accents='unicode', 
                      analyzer='word', 
                      token_pattern=r'\w{1,}',
                      ngram_range=(1, 3), 
                      use_idf=1,
                      smooth_idf=1,   # add 1 to prevent zero division
                      sublinear_tf=1, 
                      stop_words = 'english')

data_tfv = tfv.fit_transform(submission_corpus.title)
data = pd.DataFrame(data_tfv.toarray(), columns=tfv.get_feature_names())
data.index = submission_corpus.subreddit
data.head()

0         1  1 3  1 million  1 year        10     10 15  \
subreddit                                                                  
2007scape      0.0  0.027984  0.0        0.0     0.0  0.032308  0.072772   
3Dprinting     0.0  0.000000  0.0        0.0     0.0  0.000000  0.000000   
AmItheAsshole  0.0  0.000000  0.0        0.0     0.0  0.000000  0.000000   
AskMen         0.0  0.000000  0.0        0.0     0.0  0.000000  0.000000   
AskReddit      0.0  0.035038  0.0        0.0     0.0  0.000000  0.000000   

               10 15 year  10 year  10 year ago       100  1000     10000  \
subreddit                                                                   
2007scape        0.072772      0.0          0.0  0.033148   0.0  0.000000   
3Dprinting       0.000000      0.0          0.0  0.000000   0.0  0.000000   
AmItheAsshole    0.000000      0.0          0.0  0.000000   0.0  0.000000   
AskMen           0.000000      0.0          0.0  0.000000   0.0  0.000000   
AskReddit        0.000000      0.0          0.0  0.041505   0.0  0.056096   

               100k  100m  1080p  10k  10th        11       110  11th   12  \
subreddit                                                                    
2007scape       0.0   0.0    0.0  0.0   0.0  0.053867  0.000000   0.0  0.0   
3Dprinting      0.0   0.0    0.0  0.0   0.0  0.000000  0.000000   0.0  0.0   
AmItheAsshole   0.0   0.0    0.0  0.0   0.0  0.000000  0.000000   0.0  0.0   
AskMen          0.0   0.0    0.0  0.0   0.0  0.000000  0.068374   0.0  0.0   
AskReddit       0.0   0.0    0.0  0.0   0.0  0.000000  0.000000   0.0  0.0   

                     13  13 year  13 year old   14  14 year        15  \
subreddit                                                               
2007scape      0.000000      0.0          0.0  0.0      0.0  0.041236   
3Dprinting     0.000000      0.0          0.0  0.0      0.0  0.038905   
AmItheAsshole  0.000000      0.0          0.0  0.0      0.0  0.000000   
AskMen         0.047003      0.0          0.0  0.0      0.0  0.000000   
AskReddit      0.000000      0.0          0.0  0.0      0.0  0.000000   

                15 year  150  156        16  16 year  16 year old        17  \
subreddit                                                                     
2007scape      0.065795  0.0  0.0  0.000000      0.0          0.0  0.000000   
3Dprinting     0.000000  0.0  0.0  0.000000      0.0          0.0  0.071627   
AmItheAsshole  0.000000  0.0  0.0  0.000000      0.0          0.0  0.000000   
AskMen         0.000000  0.0  0.0  0.000000      0.0          0.0  0.000000   
AskReddit      0.000000  0.0  0.0  0.049186      0.0          0.0  0.000000   

                     18   19  1976  1980  1st         2  2 day  2 hour  \
subreddit                                                                
2007scape      0.000000  0.0   0.0   0.0  0.0  0.044113    0.0     0.0   
3Dprinting     0.000000  0.0   0.0   0.0  0.0  0.000000    0.0     0.0   
AmItheAsshole  0.031683  0.0   0.0   0.0  0.0  0.027342    0.0     0.0   
AskMen         0.000000  0.0   0.0   0.0  0.0  0.000000    0.0     0.0   
AskReddit      0.000000  0.0   0.0   0.0  0.0  0.023768    0.0     0.0   

               2 week  2 year        20  20 year  20 year ago  20 years  \
subreddit                                                                 
2007scape         0.0     0.0  0.000000      0.0          0.0       0.0   
3Dprinting        0.0     0.0  0.000000      0.0          0.0       0.0   
AmItheAsshole     0.0     0.0  0.000000      0.0          0.0       0.0   
AskMen            0.0     0.0  0.036344      0.0          0.0       0.0   
AskReddit         0.0     0.0  0.034365      0.0          0.0       0.0   

                    200  2000  2004  2006      2007  2008  2010  2011  2012  \
subreddit                                                                     
2007scape      0.055364   0.0   0.0   0.0  0.072772   0.0   0.0   0.0   0.0   
3Dprinting     0.000000   0.0   0.0   0.0  0.0000

In [28]:
data = data.transpose()
data.head()

subreddit,2007scape,3Dprinting,AmItheAsshole,AskMen,AskReddit,BestofRedditorUpdates,CasualUK,Cringetopia,CryptoCurrency,Damnthatsinteresting,DnD,EscapefromTarkov,Games,Genshin_Impact,GlobalOffensive,HolUp,Home,IdiotsInCars,Jokes,LivestreamFail,MadeMeSmile,MaliciousCompliance,MechanicalKeyboards,Minecraft,Music,NoStupidQuestions,OnePiece,Piracy,ProgrammerHumor,PublicFreakout,StarWars,Superstonk,ThatsInsane,Tinder,TooAfraidToAsk,TrueOffMyChest,Twitch,TwoXChromosomes,Unexpected,VALORANT,WTF,Wellthatsucks,WhitePeopleTwitter,anime,antiwork,apexlegends,aww,buildapc,discordapp,dndnext,dyinglight,euphoria,europe,explainlikeimfive,facepalm,ffxiv,formula1,funny,gaming,gifs,greentext,halo,instant_regret,interestingasfuck,leagueoflegends,lostarkgame,manga,memes,mildlyinfuriating,movies,nba,news,nextfuckinglevel,nfl,nottheonion,oddlyterrifying,olympics,ontario,pathofexile,pcgaming,pcmasterrace,personalfinance,pics,pokemon,politics,relationship_advice,science,soccer,sysadmin,technology,teenagers,television,therewasanattempt,tifu,todayilearned,unpopularopinion,videos,wallstreetbets,worldnews,wow
0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.043176,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.073591,0.0,0.00000,0.047584,0.00000,0.000000,0.000000,0.0,0.041120,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.080399,0.000000,0.075813,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.058251,0.0,0.046441,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.048235,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.048723,0.0,0.000000,0.000000,0.0,0.000000,0.073114,0.0,0.0,0.052558,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
1,0.027984,0.0,0.0,0.0,0.035038,0.045343,0.024092,0.0,0.000000,0.023611,0.0,0.0,0.0,0.039546,0.025937,0.0,0.0,0.027837,0.0,0.03082,0.000000,0.02732,0.037864,0.029256,0.0,0.021922,0.0,0.028994,0.038107,0.0,0.0,0.0,0.024928,0.031262,0.0,0.0,0.000000,0.023118,0.000000,0.050276,0.032798,0.026006,0.0,0.0,0.023603,0.024735,0.031075,0.0,0.039171,0.0,0.048944,0.048404,0.0,0.031055,0.0,0.024759,0.040968,0.0,0.02752,0.032535,0.036635,0.024141,0.0,0.031972,0.024309,0.037989,0.045887,0.0,0.000000,0.034101,0.032862,0.0,0.039425,0.028682,0.0,0.0,0.0,0.022345,0.036081,0.02246,0.0,0.000000,0.0,0.029894,0.021472,0.0,0.015065,0.030205,0.0,0.0,0.000000,0.023206,0.0,0.0,0.016806,0.0,0.0,0.025301,0.02012,0.0
1 3,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.071422,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.063492,0.080792,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.059881,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
1 million,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.08015,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.072746,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.060347,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
1 year,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.062370,0.0000

# 2. Topic Modeling with LDA

### Latent Dirichlet Allocation (LDA) 

In [32]:
from gensim import matutils, models
import scipy.sparse

corpus = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data))
id2word = dict((v,k) for k,v in tfv.vocabulary_.items())

In [33]:
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=10)
lda.print_topics()

[(0,
  '0.000*"like" + 0.000*"make" + 0.000*"think" + 0.000*"t" + 0.000*"good" + 0.000*"look" + 0.000*"want" + 0.000*"guy" + 0.000*"anon" + 0.000*"im"'),
 (1,
  '0.000*"s" + 0.000*"say" + 0.000*"make" + 0.000*"people" + 0.000*"t" + 0.000*"new" + 0.000*"use" + 0.000*"year" + 0.000*"gme" + 0.000*"time"'),
 (2,
  '0.001*"make" + 0.001*"like" + 0.001*"s" + 0.001*"people" + 0.001*"think" + 0.001*"t" + 0.001*"say" + 0.001*"new" + 0.001*"game" + 0.001*"good"')]

In [34]:
lda = models.LdaModel(corpus=corpus, num_topics=4, id2word=id2word, passes=10)
lda.print_topics()

[(0,
  '0.000*"cat" + 0.000*"husband" + 0.000*"make" + 0.000*"joy" + 0.000*"t" + 0.000*"snow" + 0.000*"taste" + 0.000*"dog" + 0.000*"cake" + 0.000*"daughter"'),
 (1,
  '0.000*"website" + 0.000*"reddit action rampant" + 0.000*"action rampant coronavirus" + 0.000*"coronavirus misinformation" + 0.000*"rampant coronavirus" + 0.000*"misinformation website" + 0.000*"action rampant" + 0.000*"reddit action" + 0.000*"rampant coronavirus misinformation" + 0.000*"coronavirus misinformation website"'),
 (2,
  '0.000*"game" + 0.000*"player" + 0.000*"team" + 0.000*"make" + 0.000*"play" + 0.000*"opinion" + 0.000*"people" + 0.000*"like" + 0.000*"think" + 0.000*"good"'),
 (3,
  '0.001*"make" + 0.001*"like" + 0.001*"s" + 0.001*"people" + 0.001*"think" + 0.001*"t" + 0.001*"say" + 0.001*"new" + 0.001*"game" + 0.001*"good"')]

### Hyperparameter tuning

In [ ]:
my_stop_words = ['like', 'just', 'https', 'com']
stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)

cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(submission_corpus.title)
data_dtmm = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmm.index = submission_corpus.subreddit
data_dtmm.head()

### Results